In [1]:
import platform
import os
import logging
import re

import pandas as pd
import cv2

from street_signs_identify.data_type.detected_image import DetectedImage
from street_signs_identify.utils.raw_image_loader import RawImageLoader
from street_signs_identify.preprocessor.masking_polygon import masking_polygon, masking_polygon_low_level
from street_signs_identify.preprocessor.subtract_bounding_box import subtract_bounding_box
from street_signs_identify.preprocessor.proportional_adjustment import proportional_adjustment
from street_signs_identify.detector.craft_detector import CRAFTDetector
from street_signs_identify.detector.fr_detector import FRDetector
from street_signs_identify.detector.frc_detector import TowStepDetector
import street_signs_identify.utils.check_overlap as co

LOGGER = logging.getLogger(__name__)

In [2]:
# Creat mask to some predefined categories
masking = True

# Select detector type: one of ["f", "c", "fc", "none"]
detector_type = "fc"

# Subtract bounding boxes to some given categories
subtract_pred = True # remove symbols

# Drop bounding boxes that the aspect ration is below the threshold
aspect_ratio_threshold = 1.5

# Drop bounding boxes (the word contains the symbol *)
drop_star = True

# Adjust the height of bounding boxes according to the ratio
proportional_ratio = 2./3

# 
iou_threshold = .3
# use_quantile_for_iou = False



config = f"{masking}_{detector_type}_{subtract_pred}_{aspect_ratio_threshold:.2f}_{drop_star}_{proportional_ratio:.2f}_{iou_threshold:.2f}"
config = re.sub("\.", "p", config)
print(config)

True_fc_True_1p50_True_0p67_0p30


In [3]:
def get_detector(detector_type:str, frc_model_path:str= None):
    if detector_type=="f":
        assert frc_model_path
        return FRDetector(model_ckpt=frc_model_path)
    elif detector_type=="c":
        return CRAFTDetector()
    elif detector_type=="fc":
        assert frc_model_path
        f = FRDetector(model_ckpt=frc_model_path)
        c = CRAFTDetector()
        return TowStepDetector(f, c)
    else:
        raise NotImplementedError(f"not supported type {detector_type}")


In [4]:
if platform.system() == "Linux":
    folder_name ="./dataset/ODmodel_all_labels"
    data_path = "./dataset/raw_data/"
    frcnn_model_path = "./street_signs_identify/detector/saved_models/frcnn.pth"
    annotation_file = "3Taoyuan_ori_all.json" #"ODmodel_all_labels.json" #"2Penghu_ori_all.json"

    output_folder = os.path.join(folder_name, config)
    gt_preview_folder = os.path.join(output_folder, "gt")
    masking_preview_folder = os.path.join(output_folder, "masking")
    aspect_ratio_filter_preview_folder = os.path.join(output_folder, "asp_filter")
    detection_preview_folder = os.path.join(output_folder, "detected_preview")
    subtract_symble_preview_folder = os.path.join(output_folder, "subtract_symbol")

    output_csv = os.path.basename(annotation_file).split('.')[0]+".csv"
    encoding = None
else:
    folder_name ="./dataset/ODmodel_all_labels"
    older_name ="./dataset/ODmodel_all_labels"
    data_path = "./dataset/raw_data/"
    frcnn_model_path = "./street_signs_identify/detector/saved_models/frcnn.pth"
    annotation_file = "3Taoyuan_ori_all.json" #"ODmodel_all_labels.json" #"2Penghu_ori_all.json"

    output_folder = os.path.join(folder_name, config)
    gt_preview_folder = os.path.join(output_folder, "gt")
    masking_preview_folder = os.path.join(output_folder, "masking")
    aspect_ratio_filter_preview_folder = os.path.join(output_folder, "asp_filter")
    detection_preview_folder = os.path.join(output_folder, "detected_preview")
    subtract_symble_preview_folder = os.path.join(output_folder, "subtract_symbol")

    output_csv = os.path.basename(annotation_file).split('.')[0]+".csv"
    encoding = 'big5'

os.makedirs(output_folder, exist_ok=True)
os.makedirs(gt_preview_folder, exist_ok=True)
os.makedirs(masking_preview_folder, exist_ok=True)
os.makedirs(detection_preview_folder, exist_ok=True)
os.makedirs(subtract_symble_preview_folder, exist_ok=True)

image_loader = RawImageLoader(data_path, annotation_file)
detector = get_detector(detector_type, frcnn_model_path)

100%|██████████| 1833/1833 [00:06<00:00, 268.07it/s]


In [5]:
# remove lower case letters 
image_loader._info["word"] = image_loader._info["word"].str.replace("[a-z]", "", regex=True)

# remove upper case letters 
image_loader._info["word"] = image_loader._info["word"].str.replace("[A-Z]", "", regex=True)

# remove '\n'
image_loader._info["word"] = image_loader._info["word"].str.replace("[\n]", "", regex=True)

# remove ‧
image_loader._info["word"] = image_loader._info["word"].str.replace("‧", "", regex=True)

# remove .
image_loader._info["word"] = image_loader._info["word"].str.replace("\.", "", regex=True)

# remove /
image_loader._info["word"] = image_loader._info["word"].str.replace("/", "", regex=True)


image_loader._info

,filename,size,polygon,category,word,no_star,aspect_ratio,x0,y0,x1,y1
0,220707_023525413_Camera_3.jpg,267955,"POLYGON ((101 932, 87 1002, 269 1042, 280 986,...",1,木墊貨專區,True,1.754545,87,932,280,1042
1,220707_023525413_Camera_3.jpg,267955,"POLYGON ((243 999, 241 1012, 253 1014, 250 102...",A_3,,True,0.931034,241,994,268,1023
2,220707_023525413_Camera_3.jpg,267955,"POLYGON ((74 1068, 261 1100, 270 1043, 88 1003...",1,,True,2.020619,74,1003,270,1100
3,220707_060950255_Camera_1.jpg,269094,"POLYGON ((2191 772, 2198 852, 2450 827, 2450 7...",3,南崁竹園,True,2.376147,2191,743,2450,852
4,220707_060950255_Camera_1.jpg,269094,"POLYGON ((2203 928, 2198 850, 2450 825, 2451 9...",1,第一航廈 1,True,2.456311,2198,825,2451,928
...,...,...,...,...,...,...,...,...,...,...,...
190,220707_053755977_Camera_1.jpg,316398,"POLYGON ((2268 1059, 2274 1066, 2259 1066, 225...",A_3,,True,0.937500,2259,1053,2289,1085
191,220707_053755977_Camera_1.jpg,316398,"POLYGON ((2284 1006, 2287 1043, 2327 1041, 232...",C_6,,True,1.075000,2284,1003,2327,1043
192,220707_053755977_Camera_1.jpg,316398,"POLYGON ((2246 963, 2248 1005, 2451 990, 2451 ...",1,出境大廳,True,3.253968,2246,942,2451,1005
193,220707_053755977_Camera_1.jpg,316398,"POLYGON ((2251 1050, 2451 1040, 2451 991, 2247...",1,1號停車場 1,True,3.457627,2247,991,2451,1050


In [6]:
label_df = pd.DataFrame(columns=["filename", "category", "words", "src_file", "gt_index", "IOU"])
global_index = 0

for image_name in image_loader._image_names:
    drop_index_list = []
    img_gt = image_loader[image_name]
    preview_img = img_gt.draw_all_box()
    cv2.imwrite(os.path.join(gt_preview_folder, image_name), cv2.cvtColor(preview_img, cv2.COLOR_RGB2BGR))
    img_gt_backup = DetectedImage(img_gt.ref_image, img_gt.info.copy())
    # preprocess on GT

    # masking symble
    if masking:
        subtract_category = ["4", "5", "C_X"] + [f"A_{i}" for i in range(1,10)] + [f"C_{i}" for i in range(6,15)]
        masking_index_b = img_gt.info["category"].isin(subtract_category)
        masking_index = img_gt.info.index[masking_index_b]
        img_gt = masking_polygon(img_gt, *masking_index, color=(0,255,255))

    # find bounding box that violate pre-defined aspect_ratio
    if aspect_ratio_threshold>0:
        index_1236 = img_gt.info["category"].isin(["1","2","3","6"])
        drop_index = img_gt.info["aspect_ratio"]<aspect_ratio_threshold
        drop_index = drop_index & index_1236
        drop_index_list.append(drop_index)

    # find word that contains * symble
    if drop_star:
        drop_index = img_gt.info["no_star"] == False
        drop_index_list.append(drop_index)

    # drop bounding box process start
    if drop_index_list:
        # accumulate all drop index from above
        drop_index = drop_index_list[0]
        for new_drop_index in drop_index_list[1:]:
            drop_index |= new_drop_index
        print(f"{image_name} drop {sum(drop_index)}/{len(drop_index)}")

        # drop bounding box here
        if masking:
            # drop all drop_index in GT and create corresponding mask
            masking_index = img_gt.info.index[drop_index]
            img_gt = masking_polygon(img_gt, *masking_index, color=(0,255,255))
            preview_img = img_gt.draw_all_box()
            cv2.imwrite(os.path.join(masking_preview_folder, image_name), cv2.cvtColor(preview_img, cv2.COLOR_RGB2BGR))  
        else:
            # drop all drop_index in GT
            img_gt.info = img_gt.info.loc[~drop_index]

    if img_gt.info.empty:
        LOGGER.warning("%s has no GT survive", image_name)
        continue

    img_pred = detector(img_gt.image)
    if img_pred is None:
        LOGGER.warning("%s failed to detect instance", img_gt.info.iloc[0]["filename"])
        continue
    
    if proportional_ratio>0:
        adjust_index = img_pred.info.index
        img_pred = proportional_adjustment(img_pred, *adjust_index, ratio=proportional_ratio)

    # save preview    
    preview_img = img_pred.draw_all_box()
    cv2.imwrite(os.path.join(detection_preview_folder, image_name), cv2.cvtColor(preview_img, cv2.COLOR_RGB2BGR))
    
    #! symbol subtraction w.r.t img_gt_backup
    if subtract_pred:
        subtract_category = ["7", "8", "C_X"] + [f"A_{i}" for i in range(1,10)] + [f"B_{i}" for i in range(1,6)] + [f"C_{i}" for i in range(6,15)]
        img_pred = subtract_bounding_box(img_pred, *subtract_category, ref_ImageHandler=img_gt_backup)
        preview_img = img_pred.draw_all_box()
        cv2.imwrite(os.path.join(subtract_symble_preview_folder, image_name), cv2.cvtColor(preview_img, cv2.COLOR_RGB2BGR))
    
    overlapping_info = co.check_overlapping_area(img_gt.info, img_pred.info)
    if overlapping_info is None:
        LOGGER.warning("Detected instance and GT are all disjoint: %s", img_gt.info.iloc[0]["filename"])
        continue
    img_pred = img_pred.keeps_index(overlapping_info["index_2nd"])

    # print(overlapping_info)
    for row_idx, row in overlapping_info.iterrows():
        if row["IOU"] < iou_threshold:
            continue
        gt_index, detected_index = row["index_1st"], row["index_2nd"]

        if img_gt.info.loc[gt_index, "category"] not in ["1", "2", "3", "6", "7", "8"] + [f"B_{i}" for i in range(1,6)]:
            continue

        detected_instance = img_pred[detected_index]
        if not detected_instance.legal:
            continue

        label_df.loc[global_index, "filename"] = f"{global_index}.jpg"
        label_df.loc[global_index, "src_file"] = image_name
        label_df.loc[global_index, "category"] = img_gt.info.loc[gt_index, "category"]
        label_df.loc[global_index, "words"] = img_gt.info.loc[gt_index, "word"]
        label_df.loc[global_index, "IOU"] = row["IOU"]
        label_df.loc[global_index, "gt_index"] = int(gt_index)
        
        cropped_img = detected_instance.image
        cv2.imwrite(os.path.join(output_folder, f"{global_index}.jpg"), cv2.cvtColor(cropped_img, cv2.COLOR_RGB2BGR))

        global_index+=1
label_df.to_csv(os.path.join(output_folder, output_csv), encoding=encoding, index=False)


220707_023525413_Camera_3.jpg drop 0/2
220707_060950255_Camera_1.jpg drop 0/4
220707_025342625_Camera_1.jpg drop 3/5


Detected instance and GT are all disjoint: 220707_025342625_Camera_1.jpg
220707_025437260_Camera_1.jpg has no GT survive


220707_025437260_Camera_1.jpg drop 0/0
220707_025442332_Camera_1.jpg drop 0/4
220707_025443788_Camera_1.jpg drop 1/4


Detected instance and GT are all disjoint: 220707_025443788_Camera_1.jpg


220707_025445172_Camera_1.jpg drop 2/4


...

220707_024150191_Camera_1.jpg drop 0/0


220707_024624272_Camera_1.jpg has no GT survive


220707_024624272_Camera_1.jpg drop 8/8
220707_025146257_Camera_1.jpg drop 0/6
220707_060100838_Camera_1.jpg drop 4/11


Detected instance and GT are all disjoint: 220707_060100838_Camera_1.jpg


220707_060159380_Camera_1.jpg drop 0/2
220707_053755977_Camera_1.jpg drop 1/5


In [7]:
label_df

,filename,category,words,src_file,gt_index,IOU
0,0.jpg,1,木墊貨專區,220707_023525413_Camera_3.jpg,0,0.314522
1,1.jpg,3,南崁竹園,220707_060950255_Camera_1.jpg,0,0.390721
2,2.jpg,3,南崁竹園,220707_060950255_Camera_1.jpg,0,0.391295
3,3.jpg,3,南崁竹園,220707_060950255_Camera_1.jpg,0,0.40574
4,4.jpg,1,第一航廈 1,220707_060950255_Camera_1.jpg,1,0.375029
5,5.jpg,1,二航入境2,220707_060950255_Camera_1.jpg,2,0.354194
6,6.jpg,1,30分鐘內,220707_054808223_Camera_1.jpg,5,0.480934
